In [89]:
from __future__ import division, print_function

import json
import math
import os
import urllib
from pathlib import Path
from typing import Iterable, List, Optional, Tuple

import pandas as pd
import requests

In [90]:
# Configure several constansts depending on a user's environment

if "username" in str(Path(".").absolute()):
    GOOGLE_MAPS_API_KEY = os.environ["MY_GM_API_KEY"]
    DATA_PATH = "."
    DISTANCE_MATRIX_CACHE_FILE = "distance.csv"
else:
    # The Google Maps API key
    GOOGLE_MAPS_API_KEY = "personal_key"
    # The data path
    DATA_PATH = "."
    # Whether to cache a distance matrix to a file
    DISTANCE_MATRIX_CACHE_FILE = None

In [91]:
# import data

deliveries = pd.read_csv(f"{DATA_PATH}/Deliveries_210315.csv")
# deliveries = pd.read_csv(f"{DATA_PATH}/Deliveries_210624.csv")
# deliveries = pd.read_csv(f"{DATA_PATH}/Deliveries_210625.csv")
deliveries["Full Address"] = (
    deliveries["Address 1"]
    + ", "
    + deliveries["City"]
    + ", "
    + deliveries["State"]
    + ", "
    + deliveries["Zip"].astype(str)
)
deliveries

,#,Address 1,City,State,Zip,Open Hr,Open Min,Close Hr,Close Min,Pallets,Weight,Loading,Full Address
0,0,812 Union St,Montebello,CA,90640,8,480,24,1440,0,0,0,"812 Union St, Montebello, CA, 90640"
1,1,768 Merchant St,Los Angeles,CA,90021,7,420,15,900,30,29944,60,"768 Merchant St, Los Angeles, CA, 90021"
2,2,14371 Industry Cr,La Mirada,CA,90638,7,420,20,1200,3,5832,60,"14371 Industry Cr, La Mirada, CA, 90638"
3,3,4501 E. 49th St,Vernon,CA,90058,7,420,13,780,3,1098,60,"4501 E. 49th St, Vernon, CA, 90058"
4,4,2612 Shenandoah Way,San Bernardino,CA,92407,19,1140,24,1440,1,824,60,"2612 Shenandoah Way, San Bernardino, CA, 92407"
5,5,1500 Eastridge Ave.,Riverside,CA,92507,1,60,24,1440,2,2721,60,"1500 Eastridge Ave., Riverside, CA, 92507"
6,6,1500 Eastridge Ave.,Riverside,CA,92507,1,60,24,1440,2,3834,1,"1500 Eastridge Ave., Riverside, CA, 92507"
7,7,1500 Eastridge Ave.,Riverside,CA,92507,1,60,24,1440,1,1875,1,"1500 Eastridge Ave., Riverside, CA, 92507"
8,8,1500 Eastridge Ave.,Riverside,CA,92507,1,60,24,1440,2,2736,1,"1500 Eastridge Ave., Riverside, CA, 92507"
9,9,6079 Rickenbacker Rd,Commerce,CA,90040,8,480,14,840,18,18659,60,"6079 Rickenbacker Rd, Commerce, CA, 90040"


In [92]:
# Convert data to format that's consumbale by Google Maps API

addresses = (
    deliveries["Full Address"].str.replace(", ", "+").str.replace(" ", ",").tolist()
)
addresses

['812,Union,St+Montebello+CA+90640',
 '768,Merchant,St+Los,Angeles+CA+90021',
 '14371,Industry,Cr+La,Mirada+CA+90638',
 '4501,E.,49th,St+Vernon+CA+90058',
 '2612,Shenandoah,Way+San,Bernardino+CA+92407',
 '1500,Eastridge,Ave.+Riverside+CA+92507',
 '1500,Eastridge,Ave.+Riverside+CA+92507',
 '1500,Eastridge,Ave.+Riverside+CA+92507',
 '1500,Eastridge,Ave.+Riverside+CA+92507',
 '6079,Rickenbacker,Rd+Commerce+CA+90040',
 '5900,East,Slauson,Ave+Los,Angeles+CA+90040',
 '6032,Triangle,Dr+Commerce+CA+90040',
 '6032,Triangle,Dr+Commerce+CA+90040',
 '15750,Meridian,Parkway+Riverside+CA+92518',
 '12500,E.,Slauson,Ave+Santa,Fe,Springs+CA+90670']

In [93]:
# Convert columns to lists to use in model

time = deliveries[["Open Min", "Close Min"]].to_numpy().tolist()
service = deliveries["Loading"].tolist()
demand_qty = deliveries["Pallets"].tolist()
demand_wt = deliveries["Weight"].tolist()

In [94]:
time

[[480, 1440],
 [420, 900],
 [420, 1200],
 [420, 780],
 [1140, 1440],
 [60, 1440],
 [60, 1440],
 [60, 1440],
 [60, 1440],
 [480, 840],
 [480, 1020],
 [480, 840],
 [480, 840],
 [300, 660],
 [480, 780]]

In [95]:
def checkType(a_list):
    for element in a_list:
        if isinstance(element, int):
            print("It's an Integer")
        if isinstance(element, str):
            print("It's an string")
        if isinstance(element, float):
            print("It's an floating number")


# numbers = [1, 2, 3]
# checkType(numbers)

In [96]:
def create_data():
    # Creates the data.
    data = {}
    data["API_key"] = GOOGLE_MAPS_API_KEY
    data["addresses"] = addresses
    return data

In [97]:
# Create distance matrix


def create_distance_matrix(data):
    addresses = data["addresses"]
    API_key = data["API_key"]
    # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests.
    max_elements = 100
    num_addresses = len(addresses)  # 16 in this example.
    # Maximum number of rows that can be computed per request (6 in this example).
    max_rows = max_elements // num_addresses
    # num_addresses = q * max_rows + r (q = 2 and r = 4 in this example).
    q, r = divmod(num_addresses, max_rows)
    dest_addresses = addresses
    distance_matrix = []
    # Send q requests, returning max_rows rows per request.
    for i in range(q):
        origin_addresses = addresses[i * max_rows : (i + 1) * max_rows]
        response = send_request(origin_addresses, dest_addresses, API_key)
        distance_matrix += build_distance_matrix(response)

    # Get the remaining remaining r rows, if necessary.
    if r > 0:
        origin_addresses = addresses[q * max_rows : q * max_rows + r]
        response = send_request(origin_addresses, dest_addresses, API_key)
        distance_matrix += build_distance_matrix(response)
    return distance_matrix

In [98]:
def send_request(origin_addresses, dest_addresses, API_key):
    """Build and send request for the given origin and destination addresses."""

    def build_address_str(addresses):
        # Build a pipe-separated string of addresses
        address_str = ""
        for i in range(len(addresses) - 1):
            address_str += addresses[i] + "|"
        address_str += addresses[-1]
        return address_str

    request = "https://maps.googleapis.com/maps/api/distancematrix/json?units=imperial"
    origin_address_str = build_address_str(origin_addresses)
    dest_address_str = build_address_str(dest_addresses)
    request = (
        request
        + "&origins="
        + origin_address_str
        + "&destinations="
        + dest_address_str
        + "&key="
        + API_key
    )
    jsonResult = urllib.request.urlopen(request).read()
    response = json.loads(jsonResult)
    return response

In [99]:
def build_distance_matrix(response):
    distance_matrix = []
    for row in response["rows"]:
        row_list = [
            math.ceil(row["elements"][j]["duration"]["value"] / 60)
            for j in range(len(row["elements"]))
        ]
        distance_matrix.append(row_list)
    return distance_matrix

In [100]:
def prepare_distance_matrix(
    distance_matrix_cache_file: Optional[str] = None,
) -> List[List[int]]:
    # Cache file for distance matrix
    cache_file = distance_matrix_cache_file and Path(distance_matrix_cache_file)
    if cache_file and cache_file.exists():
        # Load the saved distance matrix if the cache exists
        return pd.read_csv(cache_file, header=None).to_numpy().tolist()

    # Create the data
    data = create_data()
    distance_matrix = create_distance_matrix(data)

    if cache_file:
        # Save the matrix to the cache file to save Google Maps API calls
        df = pd.DataFrame(distance_matrix)
        df.to_csv(cache_file, index=False, header=False)

    return distance_matrix

In [101]:
# Save distance matrix

distance = prepare_distance_matrix(DISTANCE_MATRIX_CACHE_FILE)
distance

[[0, 17, 16, 14, 65, 65, 65, 65, 65, 8, 9, 6, 6, 65, 14],
 [17, 0, 25, 14, 66, 67, 67, 67, 67, 17, 17, 16, 16, 66, 26],
 [15, 27, 0, 25, 61, 55, 55, 55, 55, 20, 20, 17, 17, 55, 19],
 [15, 17, 25, 0, 66, 67, 67, 67, 67, 10, 9, 13, 13, 67, 27],
 [62, 65, 62, 63, 0, 30, 30, 30, 30, 66, 66, 64, 64, 30, 61],
 [66, 68, 57, 66, 32, 0, 0, 0, 0, 69, 69, 68, 68, 10, 65],
 [66, 68, 57, 66, 32, 0, 0, 0, 0, 69, 69, 68, 68, 10, 65],
 [66, 68, 57, 66, 32, 0, 0, 0, 0, 69, 69, 68, 68, 10, 65],
 [66, 68, 57, 66, 32, 0, 0, 0, 0, 69, 69, 68, 68, 10, 65],
 [9, 18, 19, 9, 67, 67, 67, 67, 67, 0, 3, 5, 5, 67, 20],
 [8, 18, 18, 8, 67, 67, 67, 67, 67, 2, 0, 5, 5, 67, 19],
 [7, 20, 17, 13, 66, 66, 66, 66, 66, 6, 6, 0, 0, 66, 17],
 [7, 20, 17, 13, 66, 66, 66, 66, 66, 6, 6, 0, 0, 66, 17],
 [66, 69, 56, 67, 32, 10, 10, 10, 10, 69, 69, 68, 68, 0, 65],
 [14, 28, 18, 26, 62, 63, 63, 63, 63, 19, 20, 16, 16, 63, 0]]

In [107]:
# Create dataset

data = {}
# Weights (i.e costs, such as time) between nodes
# For example, cost 6 at index (0, 1) represents a cost between node 0 and 1 is 6
data["weights"] = distance
# Times it takes for a vehicle to leave from the time it stops at each node
data["service_times"] = service
# Demands corresponding to the quantity (such as volume) of items to be delivered
data["demands_qty"] = demand_qty
# Demands corresponding to the quantity (such as volume) of items to be delivered
data["demands_wt"] = demand_wt
# Time windows for nodes which are considered as requested times for a visit
# Vehicles must visit a node within its time window
data["time_windows"] = time

# Vehicle capacities
data["vehicle_capacities_qty"] = [30, 30, 30, 12, 30, 30, 30]
# data["vehicle_capacities_wt"] = [42000, 42000, 42000, 13000, 42000, 42000, 42000]
# Temporarily set weight capacity to unrealistically large values so that our VRP solver
# can satisfy weight contraints
data["vehicle_capacities_wt"] = [44500, 44500, 44500, 13000, 44500, 44500, 44500]

# Index of the start node
data["depot"] = 0

In [103]:
data

{'demands_qty': [0, 30, 3, 3, 1, 2, 2, 1, 2, 18, 14, 18, 1, 15, 12],
 'demands_wt': [0,
  29944,
  5832,
  1098,
  824,
  2721,
  3834,
  1875,
  2736,
  18659,
  13924,
  25349,
  231,
  42750,
  19806],
 'depot': 0,
 'service_times': [0, 60, 60, 60, 60, 60, 1, 1, 1, 60, 60, 60, 1, 60, 60],
 'time_windows': [[480, 1440],
  [420, 900],
  [420, 1200],
  [420, 780],
  [1140, 1440],
  [60, 1440],
  [60, 1440],
  [60, 1440],
  [60, 1440],
  [480, 840],
  [480, 1020],
  [480, 840],
  [480, 840],
  [300, 660],
  [480, 780]],
 'vehicle_capacities_qty': [30, 30, 30, 30, 30, 30],
 'vehicle_capacities_wt': [44500, 44500, 44500, 44500, 44500, 44500],
 'weights': [[0, 17, 16, 14, 65, 65, 65, 65, 65, 8, 9, 6, 6, 65, 14],
  [17, 0, 25, 14, 66, 67, 67, 67, 67, 17, 17, 16, 16, 66, 26],
  [15, 27, 0, 25, 61, 55, 55, 55, 55, 20, 20, 17, 17, 55, 19],
  [15, 17, 25, 0, 66, 67, 67, 67, 67, 10, 9, 13, 13, 67, 27],
  [62, 65, 62, 63, 0, 30, 30, 30, 30, 66, 66, 64, 64, 30, 61],
  [66, 68, 57, 66, 32, 0, 0, 0,

In [104]:
# solution = "gls"
solution = "verbose"
graph = "Yes"

In [108]:
"""
Solve the Capacitated Vehicle Routing Problem with Time Windows (CVRPTW).
"""

import argparse
from dataclasses import dataclass
from typing import Callable

# import pygraphviz as pgv
# import graphviz
# import yaml
from ortools.constraint_solver.pywrapcp import (
    Assignment,
    DefaultRoutingSearchParameters,
    RoutingDimension,
    RoutingIndexManager,
    RoutingModel,
)
from ortools.constraint_solver.routing_enums_pb2 import (
    FirstSolutionStrategy,
    LocalSearchMetaheuristic,
)

TransitCallback = Callable[[int, int], int]
UnaryTransitCallback = Callable[[int], int]

MIN_PER_HOUR = 60

delay = 30

@dataclass
class Node:
    idx: int  # Index of the node
    address: str  # Full address of the node
    arrival_time: int  # Actual arrival time (minutes)
    start_time: int  # Actual start time of the appointment
    departure_time: int  # Actual departure time (minutes)
    quantity: int  # Currently consumed quantity capacity
    weight: int  # Currently consumed weight capacity

    def format(self) -> str:
        # Convert minutes to clock time
        arrival_time = self.get_clock(self.arrival_time)
        start_time = self.get_clock(self.start_time)
        departure_time = self.get_clock(self.departure_time)

        # Round up the arrival time to the nearest full hour to get an appointment time
        appointment_time = (
            self.get_clock(self.start_time // MIN_PER_HOUR * MIN_PER_HOUR)
            if self.idx != 0
            # Depot does not need an appointment time
            else "--:--"
        )
        # Grace time between arrival time and appointment start time
        slack = self.start_time - self.arrival_time

        return (
            "["
            f"Node {self.idx:2d}:"
            f" Quantity({self.quantity:2d})"
            f" Weight({self.weight:5d})"
            f" Arrival({arrival_time})"
            f" Slack({slack:3d})"
            f" Appointment({appointment_time})"
            f" Time({start_time}, {departure_time})"
            # f" Window({time_min}, {time_max})"
            "]"
            f" {self.address}"
        )

    def get_clock(self, time: int) -> str:
        hour, minute = divmod(time, 60)
        return f"{hour:02d}:{minute:02d}"


def load_data_model():
    """
    Load the data for the problem from path.
    """
    data["num_locations"] = len(data["time_windows"])
    data["num_vehicles"] = len(data["vehicle_capacities_qty"])

    return data


def create_weight_callback(manager: RoutingIndexManager, data: dict) -> TransitCallback:
    """
    Create a callback to return the weight between points.
    """

    def weight_callback(from_index: int, to_index: int) -> int:
        """
        Return the weight between the two points.
        """

        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["weights"][from_node][to_node]

    return weight_callback


def create_demand_callback_qty(
    manager: RoutingIndexManager, data: dict
) -> UnaryTransitCallback:
    """
    Create a callback to get demands at each location.
    """

    def demand_callback_qty(from_index: int) -> int:
        """
        Return the demand.
        """

        from_node = manager.IndexToNode(from_index)
        return data["demands_qty"][from_node]

    return demand_callback_qty


def create_demand_callback_wt(
    manager: RoutingIndexManager, data: dict
) -> UnaryTransitCallback:
    """
    Create a callback to get demands at each location.
    """

    def demand_callback_wt(from_index: int) -> int:
        """
        Return the demand.
        """

        from_node = manager.IndexToNode(from_index)
        return data["demands_wt"][from_node]

    return demand_callback_wt


def add_capacity_constraints_qty(
    routing: RoutingModel,
    manager: RoutingIndexManager,
    data: dict,
    demand_callback_index_qty: int,
) -> None:
    """
    Add capacity constraints.
    """

    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index_qty,
        slack_max=0,  # null capacity slack
        vehicle_capacities=data["vehicle_capacities_qty"],  # vehicle maximum capacities
        fix_start_cumul_to_zero=True,  # start cumul to zero
        name="CapacityQty",
    )


def add_capacity_constraints_wt(
    routing: RoutingModel,
    manager: RoutingIndexManager,
    data: dict,
    demand_callback_index_wt: int,
) -> None:
    """
    Add capacity constraints.
    """

    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index_wt,
        slack_max=0,  # null capacity slack
        vehicle_capacities=data["vehicle_capacities_wt"],  # vehicle maximum capacities
        fix_start_cumul_to_zero=True,  # start cumul to zero
        name="CapacityWt",
    )


def create_time_callback(manager: RoutingIndexManager, data: dict) -> TransitCallback:
    """
    Create a callback to get total times between locations.
    """

    def time_callback(from_index: int, to_index: int) -> int:
        """
        Return the total time between the two nodes.
        """

        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)

        # Get the service time to the specified location
        serv_time = data["service_times"][from_node]
        # Get the travel times between two locations
        trav_time = data["weights"][from_node][to_node]

        return serv_time + trav_time

    return time_callback


def add_time_window_constraints(
    routing: RoutingModel,
    manager: RoutingIndexManager,
    data: dict,
    time_callback_index: int,
    individual_travel_time_penalty: int = 0,
    max_delay: int = delay,
) -> None:
    """
    Add time window constraints.
    """

    horizon = 60 * 24
    routing.AddDimension(
        time_callback_index,
        slack_max=horizon,  # allow waiting time
        capacity=horizon,  # maximum time per vehicle
        # Don't force start cumul to zero.
        fix_start_cumul_to_zero=False,
        name="Time",
    )

    time_dimension = routing.GetDimensionOrDie("Time")
    time_windows = data["time_windows"]

    # Set the time window for each node
    for loc_idx, (open_time, close_time) in enumerate(time_windows):
        cumul_var = time_dimension.CumulVar(manager.NodeToIndex(loc_idx))

        # Set open and close time
        cumul_var.SetRange(open_time, close_time)

        # Remove forbidden intervals within the time window
        for start, end in forbidden_intervals(open_time, close_time, max_delay):
            cumul_var.RemoveInterval(start, end)

    # Set the time window of the depot for each start/end node of each vehicle
    # to respect the depot's open and close time
    depot_open_time, depot_close_time = data["time_windows"][data["depot"]]
    for vehicle_id in range(len(data["vehicle_capacities_qty"])):
        for index in (routing.Start(vehicle_id), routing.End(vehicle_id)):
            time_dimension.CumulVar(index).SetRange(depot_open_time, depot_close_time)

    if individual_travel_time_penalty > 0:
        # The longer the individual travel time, the heavier the cost for that vehicle
        time_dimension.SetSpanCostCoefficientForAllVehicles(
            individual_travel_time_penalty
        )


def forbidden_intervals(
    open_time: int, close_time: int, max_delay: int = delay
) -> Iterable[Tuple[int, int]]:
    """
    Return a sequence of forbidden intervals during which vehicles should not visit
    a node.
    Since all times when vehicles should visit nodes must be aligned between every
    hour on the hour and `max_delay` minutes after that, this function constructs
    and returns a sequence of intervals other than those allowed.

    For example, if a time window of a node is [240, 480] (i.e. [4am, 8am]) and
    a given `max_delay` is 14 min, allowed "sub time windows" within the time window
    are as follows:

      [(240, 254), # 04:00 - 04:14
       (300, 314), # 05:00 - 05:14
       (360, 374), # 06:00 - 06:14
       (420, 434), # 07:00 - 07:14
       (480, 480)] # 08:00 - 08:00

    Therefore, this function returns the following forbidden intervals:

      [(255, 299), # 04:15 - 04:59
       (315, 359), # 05:15 - 05:59
       (375, 419), # 06:15 - 06:59
       (435, 479)] # 07:15 - 07:59
    """

    delay_threshold = max_delay + 1

    # Create a sequence of pairs of start and end times of forbidden intervals
    return zip(
        # Sequence of start times of forbidden intervals
        range(
            open_time + delay_threshold, close_time - delay_threshold + 1, MIN_PER_HOUR
        ),
        # Sequence of end times of forbidden intervals
        range(open_time - 1 + MIN_PER_HOUR, close_time + 1, MIN_PER_HOUR),
    )


def create_node(
    index: int,
    manager: RoutingIndexManager,
    addresses: list,
    prev_index: Optional[int],
    prev_departure_time: Optional[int],
    data: dict,
    time_dimension: RoutingDimension,
    capacity_dimension_qty: RoutingDimension,
    capacity_dimension_wt: RoutingDimension,
    assignment: Assignment,
    depot_start_time: int,
) -> Node:
    node_index = manager.IndexToNode(index)
    address = addresses[node_index]
    cumul_minutes = assignment.Min(time_dimension.CumulVar(index))
    start_time = depot_start_time + cumul_minutes
    arrival_time = start_time
    if prev_index and prev_departure_time:
        prev_node_index = manager.IndexToNode(prev_index)
        arrival_time = (
            prev_departure_time + data["weights"][prev_node_index][node_index]
        )
    departure_time = start_time + data["service_times"][node_index]
    quantity = assignment.Value(capacity_dimension_qty.CumulVar(index))
    weight = assignment.Value(capacity_dimension_wt.CumulVar(index))
    return Node(
        node_index, address, arrival_time, start_time, departure_time, quantity, weight
    )


def print_solution(
    data: dict,
    routing: RoutingModel,
    manager: RoutingIndexManager,
    assignment: Assignment,
    addresses: list,
) -> None:
    """
    Print the solution.
    """

    capacity_dimension_qty = routing.GetDimensionOrDie("CapacityQty")
    capacity_dimension_wt = routing.GetDimensionOrDie("CapacityWt")
    time_dimension = routing.GetDimensionOrDie("Time")
    total_travel_time = 0

    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        prev_index = prev_departure_time = None
        nodes = []

        while not routing.IsEnd(index):
            node = create_node(
                index,
                manager,
                addresses,
                prev_index,
                prev_departure_time,
                data,
                time_dimension,
                capacity_dimension_qty,
                capacity_dimension_wt,
                assignment,
                depot_start_time=0,
            )
            nodes.append(node)
            prev_index = index
            prev_departure_time = node.departure_time
            index = assignment.Value(routing.NextVar(index))

        node = create_node(
            index,
            manager,
            addresses,
            prev_index,
            prev_departure_time,
            data,
            time_dimension,
            capacity_dimension_qty,
            capacity_dimension_wt,
            assignment,
            depot_start_time=0,
        )
        nodes.append(node)
        dist = data["weights"]
        route = (
            "".join(
                f"{node.format()}\n  --({dist[node.idx][next_node.idx]:2d})--> "
                for node, next_node in zip(nodes, nodes[1:])
            )
            + nodes[-1].format()
        )
        finish_time = assignment.Value(time_dimension.CumulVar(index))
        travel_time = nodes[-1].arrival_time - nodes[0].departure_time
        plan_output = (
            f"Route for vehicle {vehicle_id}:\n"
            f"            {route}\n"
            f"Total quantity: {node.quantity}\n"
            f"Total weight: {node.weight}\n"
            f"Travel time: {travel_time} min\n"
            f"Finish time: {finish_time} min\n"
        )
        print(plan_output)

        total_travel_time += travel_time

    print(f"Total travel time of all routes: {total_travel_time} min")


# def draw_network_graph(data: dict, filename: str = 'network.svg', prog: str = 'dot') -> None:
#     """
#     Draw a network graph of the problem.
#     """

#     weights = data['weights']
#     demands = data['demands']
#     time_windows = data['time_windows']
#     n_loc = data['num_locations']
#     graph = pgv.AGraph(directed=False)

#     def _node(index: int) -> str:
#         if index == 0:
#             return f'{index}\nDepot'
#         return f'{index}\nDemand: {demands[index]}\nRange: {time_windows[index]}'

#     for i in range(n_loc):
#         for j in range(i + 1, n_loc):
#             weight = weights[i][j]
#             graph.add_edge(_node(i), _node(j), weight=weight, label=weight)

#     graph.draw(filename, prog=prog)

#     print(f'The network graph has been saved to {filename}.')


# def draw_route_graph(
#     data: dict,
#     routing: RoutingModel,
#     manager: RoutingIndexManager,
#     assignment: Assignment,
#     filename: str = 'route.png',
#     prog='sfdp',
# ) -> None:
#     """
#     Draw a route graph based on the solution of the problem.
#     """

#     weights = data['weights']
#     demands = data['demands']
#     time_windows = data['time_windows']
#     graph = pgv.AGraph(directed=True)

#     def _node(index: int) -> str:
#         if index == 0:
#             return f'{index}\nDepot'
#         return f'{index}\nDemand: {demands[index]}\nRange: {time_windows[index]}'

#     for vehicle_id in range(data['num_vehicles']):
#         index = routing.Start(vehicle_id)
#         while not routing.IsEnd(index):
#             node_index = manager.IndexToNode(index)
#             next_index = assignment.Value(routing.NextVar(index))
#             next_node_index = manager.IndexToNode(next_index)
#             weight = weights[node_index][next_node_index]
#             graph.add_edge(_node(node_index), _node(next_node_index), weight=weight, label=weight)
#             index = next_index

#     graph.draw(filename, prog=prog)

#     print(f'The route graph has been saved to {filename}.')


def main():
    """
    Entry point of the program.
    """

    # Instantiate the data problem
    data = load_data_model()

    # Create the Routing Index Manager and Routing Model
    manager = RoutingIndexManager(
        data["num_locations"], data["num_vehicles"], data["depot"]
    )
    routing = RoutingModel(manager)

    # Define weight of each edge
    weight_callback_index = routing.RegisterTransitCallback(
        create_weight_callback(manager, data)
    )
    routing.SetArcCostEvaluatorOfAllVehicles(weight_callback_index)

    # Add capacity constraints
    demand_callback_qty = create_demand_callback_qty(manager, data)
    demand_callback_wt = create_demand_callback_wt(manager, data)
    demand_callback_index_qty = routing.RegisterUnaryTransitCallback(
        demand_callback_qty
    )
    demand_callback_index_wt = routing.RegisterUnaryTransitCallback(demand_callback_wt)
    add_capacity_constraints_qty(routing, manager, data, demand_callback_index_qty)
    add_capacity_constraints_wt(routing, manager, data, demand_callback_index_wt)

    # Add time window constraints
    time_callback_index = routing.RegisterTransitCallback(
        create_time_callback(manager, data)
    )
    add_time_window_constraints(
        routing,
        manager,
        data,
        time_callback_index,
        # How much we add a cost proportional to the individual travel time
        individual_travel_time_penalty=1,
        # The maximum number of minutes of delay allowed
        max_delay= delay,
    )

    # Set first solution heuristic (cheapest addition)
    search_params = DefaultRoutingSearchParameters()
    # Set a reasonable time limit to avoid endless computation
    search_params.time_limit.seconds = 30
    search_params.first_solution_strategy = FirstSolutionStrategy.PATH_CHEAPEST_ARC
    if solution == "gls":
        search_params.local_search_metaheuristic = (
            LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
        )
    if solution == "verbose":
        search_params.log_search = True

    # Solve the problem
    assignment = routing.SolveWithParameters(search_params)
    if not assignment:
        print("No solution found.")
        return

    # Print the solution
    addrs = deliveries["Full Address"].tolist()
    print_solution(data, routing, manager, assignment, addrs)

    # Draw network and route graphs


#     if (graph == 'Yes'):
#         print()
#         draw_network_graph(data)
#         draw_route_graph(data, routing, manager, assignment)


if __name__ == "__main__":
    main()

Route for vehicle 0:
            [Node  0: Quantity( 0) Weight(    0) Arrival(08:00) Slack(  0) Appointment(--:--) Time(08:00, 08:00)] 812 Union St, Montebello, CA, 90640
  --( 6)--> [Node 12: Quantity( 0) Weight(    0) Arrival(08:06) Slack(  0) Appointment(08:00) Time(08:06, 08:07)] 6032 Triangle Dr, Commerce, CA, 90040
  --( 0)--> [Node 11: Quantity( 1) Weight(  231) Arrival(08:07) Slack(  0) Appointment(08:00) Time(08:07, 09:07)] 6032 Triangle Dr, Commerce, CA, 90040
  --( 7)--> [Node  0: Quantity(19) Weight(25580) Arrival(09:14) Slack(  0) Appointment(--:--) Time(09:14, 09:14)] 812 Union St, Montebello, CA, 90640
Total quantity: 19
Total weight: 25580
Travel time: 74 min
Finish time: 554 min

Route for vehicle 1:
            [Node  0: Quantity( 0) Weight(    0) Arrival(08:00) Slack(  0) Appointment(--:--) Time(08:00, 08:00)] 812 Union St, Montebello, CA, 90640
  --(65)--> [Node 13: Quantity( 0) Weight(    0) Arrival(09:05) Slack(  0) Appointment(09:00) Time(09:05, 10:05)] 15750 Mer